In [ ]:
# Code adapted from https://github.com/vhsabino/astar-paris-subway-simplified

import heapq
import pandas as pd

class priorityQueue:
    def __init__(self):
        self.cities = []

    def push(self, city, cost):
        heapq.heappush(self.cities, (cost, city))

    def pop(self):
        return heapq.heappop(self.cities)

    def border(self):
        return sorted(self.cities)

    def isEmpty(self):
        if (self.cities == []):
            return True
        else:
            return False

    def check(self):
        print(self.cities)

londres = []
londresConnect = []
linhas = pd.read_csv("linhasLondres.csv", sep = ';')
linhas = linhas.fillna(100)
linhas = (linhas - 1).astype(int)

def makearray(): #cria matrizes de distancia
    file1 = open("londres-direct.txt", 'r')
    for string in file1:
        row = string.split(',')
        for i in range(len(row)):
            row[i] = float(row[i])
        londres.append(row)
    file2 = open("londres-connect.txt", 'r')
    for string in file2:
        row = string.split(',')
        for i in range(len(row)):
            row[i] = float(row[i])
        londresConnect.append(row)

def get_g(start, goal): # funcao para calcular g(n)
    s1 = start.split('E')
    s1 = int(s1[1]) - 1
    s2 = goal.split('E')
    s2 = int(s2[1]) - 1
    if s1 > s2:
        s1, s2 = s2, s1
    g = londresConnect[s1][s2] #
    return g

def get_h(start, goal): #funcao para calcular h(n)
    s1 = start.split('E')
    s1 = int(s1[1]) - 1
    s2 = goal.split('E')
    s2 = int(s2[1]) - 1
    if s1 > s2:
        s1, s2 = s2, s1
    h = londres[s1][s2]
    return h

def string_to_info(string):
  s, color = string.split(', ')
  number = int(s.split('E')[1]) - 1
  return s,  color, number

def dist_to_time(distance):
  return (distance/40)*60

def getAvailableCities(current): #retorna todas as cidades conectadas a cidade atual
    _,c1,s1 = string_to_info(current)

    availableCities = []
    for i in range(len(londresConnect[s1])):
        if londresConnect[s1][i] != 0.0 and i > s1 and linhas[c1].isin([i]).any(): #impede de voltar para a cidade de origem
            city = "E" + str(i + 1) + ', ' + c1
            availableCities.append(city)
        if londresConnect[i][s1] != 0.0 and i <= s1 and linhas[c1].isin([i]).any():
            city = "E" + str(i + 1) + ', ' + c1
            availableCities.append(city)
        color_start = linhas.isin([s1]).any()
        color_start = color_start[color_start].index.tolist()
        if len(color_start) !=1:
          color_start.remove(c1)
          city = "E" + str(s1+1) + ', ' + color_start[0]
          availableCities.append(city)

    return availableCities

def astar(start, end):
    path = {}
    distance = {}
    time = {}
    q = priorityQueue()
    q.push(start, 0)
    distance[start] = 0
    time[start] = 0
    path[start] = None
    expandedList = []
    printoutput(start, end, path, distance, time,expandedList, q, 0)


    while (q.isEmpty() == False):
        current = q.pop()[1]
        expandedList.append(current)
        if (current == end):
            break

        availableCities = getAvailableCities(current)

        for new in availableCities:
            n,_,_ = string_to_info(new)
            c,_,_  = string_to_info(current)
            e,_,_ = string_to_info(end)

            g_cost = distance[current] + get_g(c,n)
            g_time = time[current] + dist_to_time(get_g(c,n))
            if c == n:
                g_time +=3

            #print("from " + current + " to " + new + " => " + str(g_cost))
            if (new not in time or g_time < time[new]):

                distance[new] = g_cost
                time[new] = g_time
                f_cost = g_cost + get_h(n,e)
                f_time = g_time + dist_to_time(get_h(n,e))



                #print("from " + current + " to " + new + " => " + str(f_cost))
                q.push(new,f_time)
                path[new] = current
        printoutput(start, end, path, distance,time, expandedList, q, 1)
    printoutput(start, end, path, distance,time, expandedList, q, 2)


def printoutput(start, end, path, distance,time, expandedlist, q, stage):
    finalpath = []
    i = end

    if stage == 0:
        print("\nBusca A* no mapa de londres\n")
        print("\t Percurso: " + str(start) + " => " + str(end) + "\n")
        print("====================================\n")
    elif stage > 0:
        print("Fronteira de Busca \t\t: " + str(q.border()))
        print("Estações Expandidas \t\t: " + str(expandedlist) + "  #" + str(len(expandedlist)) + "\n")
    if stage == 2:
        while (path.get(i) != None):
            finalpath.append(i)
            i = path[i]
        finalpath.append(start)
        finalpath.reverse()
        print("\n=======================================================\n")
        print("Menor caminho \t: " + str(finalpath))
        print("Numero de estações visitas \t\t\t: " + str(len(finalpath)))
        print("Distancia total percorrida \t\t\t: " + str(distance[end]) + "Km")
        print("Tempo total da viagem \t\t\t: " + str(time[end]) + "min\n\n")




def main():
    makearray() # criar vetores de distancia (tabelas)
    start = "E6, Azul" # estacao inicial
    goal = "E2, Verde" # estacao destino
    astar(start, goal)


if __name__ == "__main__":
    main()


Busca A* no mapa de londres

	 Percurso: E6, Azul => E2, Verde


Fronteira de Busca 		: [(20.25, 'E7, Azul')]
Estações Expandidas 		: ['E6, Azul']  #1

Fronteira de Busca 		: [(23.25, 'E7, Verde'), (25.5, 'E3, Azul')]
Estações Expandidas 		: ['E6, Azul', 'E7, Azul']  #2

Fronteira de Busca 		: [(25.5, 'E3, Azul'), (29.250000000000004, 'E2, Verde')]
Estações Expandidas 		: ['E6, Azul', 'E7, Azul', 'E7, Verde']  #3

Fronteira de Busca 		: [(28.5, 'E3, Vermelho'), (29.250000000000004, 'E2, Verde'), (34.05, 'E8, Azul')]
Estações Expandidas 		: ['E6, Azul', 'E7, Azul', 'E7, Verde', 'E3, Azul']  #4

Fronteira de Busca 		: [(28.5, 'E2, Vermelho'), (29.250000000000004, 'E2, Verde'), (34.05, 'E8, Azul'), (44.85, 'E4, Vermelho')]
Estações Expandidas 		: ['E6, Azul', 'E7, Azul', 'E7, Verde', 'E3, Azul', 'E3, Vermelho']  #5

Fronteira de Busca 		: [(29.250000000000004, 'E2, Verde'), (34.05, 'E8, Azul'), (41.400000000000006, 'E1, Vermelho'), (44.85, 'E4, Vermelho')]
Estações Expandidas 		: ['E6, A

In [ ]:
def main():
    makearray() # criar vetores de distancia (tabelas)
    start = "E5, Amarelo" # estacao inicial
    goal = "E7, Azul" # estacao destino
    astar(start, goal)


if __name__ == "__main__":
    main()


Busca A* no mapa de londres

	 Percurso: E5, Amarelo => E7, Azul


Fronteira de Busca 		: [(23.4, 'E4, Amarelo')]
Estações Expandidas 		: ['E5, Amarelo']  #1

Fronteira de Busca 		: [(26.4, 'E4, Vermelho'), (28.35, 'E8, Amarelo')]
Estações Expandidas 		: ['E5, Amarelo', 'E4, Amarelo']  #2

Fronteira de Busca 		: [(27.750000000000004, 'E3, Vermelho'), (28.35, 'E8, Amarelo'), (42.3, 'E14, Vermelho')]
Estações Expandidas 		: ['E5, Amarelo', 'E4, Amarelo', 'E4, Vermelho']  #3

Fronteira de Busca 		: [(28.35, 'E8, Amarelo'), (30.750000000000004, 'E3, Azul'), (39.6, 'E2, Vermelho'), (42.3, 'E14, Vermelho')]
Estações Expandidas 		: ['E5, Amarelo', 'E4, Amarelo', 'E4, Vermelho', 'E3, Vermelho']  #4

Fronteira de Busca 		: [(30.750000000000004, 'E3, Azul'), (31.35, 'E8, Azul'), (37.800000000000004, 'E9, Amarelo'), (39.6, 'E2, Vermelho'), (42.3, 'E14, Vermelho')]
Estações Expandidas 		: ['E5, Amarelo', 'E4, Amarelo', 'E4, Vermelho', 'E3, Vermelho', 'E8, Amarelo']  #5

Fronteira de Busca 		: [(3